In [1]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np

In [18]:
df = pd.read_excel("learning.xlsx",index_col = 0)
print(len(df))
df["Title"] = df["Title"].astype(str)
df["Description"] = df["Description"].astype(str)
df["StrPrimaryCategoryID"] = df["PrimaryCategoryID"].apply(lambda x: "category"+str(x))
df['Title'] = df[['StrPrimaryCategoryID','Title']].apply(lambda x: ' '.join(x), axis=1)
df['Content'] = df[['Title', 'Description','StrPrimaryCategoryID']].apply(lambda x: ' '.join(x), axis=1)
df.head()

23154


,ItemID,value,_currencyID,ViewItemURLForNaturalSearch,Title,PrimaryCategoryID,Country,Description,group,StrPrimaryCategoryID,Content
0,401682438519,1139.67,USD,https://www.ebay.com/itm/Toshiba-PVT-770RT-End...,category184515 Toshiba PVT-770RT Endorectal (b...,184515,AU,<!-- /* General Settings ::::::::::::::::::::...,Array,category184515,category184515 Toshiba PVT-770RT Endorectal (b...
1,293250724359,24.93,USD,https://www.ebay.com/itm/TOYOTA-AVENSIS-VVTI-1...,"category173865 TOYOTA AVENSIS, VVTI,1.8 MASS A...",173865,GB,WELCOME TO A1 MOTOR GROUPPART FOR SALEUp for s...,NotUS,category173865,"category173865 TOYOTA AVENSIS, VVTI,1.8 MASS A..."
2,254338051791,41.43,USD,https://www.ebay.com/itm/SIEMENS-7KG6113-3EK17...,category181708 SIEMENS 7KG6113-3EK17-0B Curren...,181708,DE,SIEMENS 7KG6113-3EK17-0B Current Transducer T...,NotUS,category181708,category181708 SIEMENS 7KG6113-3EK17-0B Curren...
3,254111396530,569.84,USD,https://www.ebay.com/itm/Toshiba-PLT-1204AX-Li...,category184515 Toshiba PLT-1204AX Linear Trans...,184515,AU,<!-- /* General Settings ::::::::::::::::::::...,Array,category184515,category184515 Toshiba PLT-1204AX Linear Trans...
4,264701751561,217.96,USD,https://www.ebay.com/itm/Aloka-Prosound-SSD-35...,category181708 Aloka Prosound SSD-3500 Plus Ul...,181708,US,"ebay template .header-wrapper,.subheader-...",Boards,category181708,category181708 Aloka Prosound SSD-3500 Plus Ul...


In [19]:
df['Title_Clean'] = df['Title'].str.replace("\r", " ")
df['Title_Clean'] = df['Title_Clean'].str.replace("\n", " ")
df['Title_Clean'] = df['Title_Clean'].str.replace("    ", " ")
df['Title_Clean'] = df['Title_Clean'].str.replace('"', '')
df['Title_Clean'] = df['Title_Clean'].str.lower()

punctuation_signs = list('?:!,;-/*"')

for punct_sign in punctuation_signs:
    df['Title_Clean'] = df['Title_Clean'].str.replace(punct_sign, '')
    
df['Title_Clean'] = df['Title_Clean'].str.replace("'s", "")

In [20]:
# Downloading punkt and wordnet from NLTK
nltk.download('punkt')
print("------------------------------------------------------------")
nltk.download('wordnet')
# Saving the lemmatizer into an object
wordnet_lemmatizer = WordNetLemmatizer()

------------------------------------------------------------


[nltk_data] Downloading package punkt to /home/kelu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/kelu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [21]:
nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['Title_Clean']
    text_words = text.split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    # Join the list
    lemmatized_text = " ".join(lemmatized_list)
    
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)

In [22]:
df['Title_Clean'] = lemmatized_text_list
df['Title_Clean']

0        category184515 toshiba pvt770rt endorectal (bi...
1        category173865 toyota avensis vvti1.8 mass air...
2        category181708 siemens 7kg61133ek170b current ...
3        category184515 toshiba plt1204ax linear transd...
4        category181708 aloka prosound ssd3500 plus ult...
                               ...                        
23149           category92078 sonosite m turbo p21x 51 mhz
23150    category184542 wireless portable ultrasound sc...
23151    category104247 oscilloscopio analogico  hitach...
23152    category184542 sonosite 180 plus hand carry ul...
23153    category185256 diagnostic ultrasound corporati...
Name: Title_Clean, Length: 23154, dtype: object

In [23]:
# Downloading the stop words list
nltk.download('stopwords')
# Loading the stop words in english
stop_words = list(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/kelu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
# Downloading the stop words list
nltk.download('stopwords')
# Loading the stop words in english
stop_words = list(stopwords.words('english'))

from tqdm.notebook import trange, tqdm

for stop_word in tqdm(stop_words, 
                         total=len(stop_words), 
                         desc="Processing records"):
    regex_stopword = r"\b" + stop_word + r"\b"
    df['Title_Clean'] = df['Title_Clean'].str.replace(regex_stopword, '')

[nltk_data] Downloading package stopwords to /home/kelu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
category_codes = {}
category_names = {}
i = 0
for k in df.group.unique().tolist():
    category_codes[k] = i
    category_names[i] = k
    i += 1

In [26]:
df['Category_Code'] = df['group']
df = df.replace({'Category_Code':category_codes})
df.head()

,ItemID,value,_currencyID,ViewItemURLForNaturalSearch,Title,PrimaryCategoryID,Country,Description,group,StrPrimaryCategoryID,Content,Title_Clean,Category_Code
0,401682438519,1139.67,USD,https://www.ebay.com/itm/Toshiba-PVT-770RT-End...,category184515 Toshiba PVT-770RT Endorectal (b...,184515,AU,<!-- /* General Settings ::::::::::::::::::::...,Array,category184515,category184515 Toshiba PVT-770RT Endorectal (b...,category184515 toshiba pvt770rt endorectal (bi...,0
1,293250724359,24.93,USD,https://www.ebay.com/itm/TOYOTA-AVENSIS-VVTI-1...,"category173865 TOYOTA AVENSIS, VVTI,1.8 MASS A...",173865,GB,WELCOME TO A1 MOTOR GROUPPART FOR SALEUp for s...,NotUS,category173865,"category173865 TOYOTA AVENSIS, VVTI,1.8 MASS A...",category173865 toyota avensis vvti1.8 mass air...,1
2,254338051791,41.43,USD,https://www.ebay.com/itm/SIEMENS-7KG6113-3EK17...,category181708 SIEMENS 7KG6113-3EK17-0B Curren...,181708,DE,SIEMENS 7KG6113-3EK17-0B Current Transducer T...,NotUS,category181708,category181708 SIEMENS 7KG6113-3EK17-0B Curren...,category181708 siemens 7kg61133ek170b current ...,1
3,254111396530,569.84,USD,https://www.ebay.com/itm/Toshiba-PLT-1204AX-Li...,category184515 Toshiba PLT-1204AX Linear Trans...,184515,AU,<!-- /* General Settings ::::::::::::::::::::...,Array,category184515,category184515 Toshiba PLT-1204AX Linear Trans...,category184515 toshiba plt1204ax linear transd...,0
4,264701751561,217.96,USD,https://www.ebay.com/itm/Aloka-Prosound-SSD-35...,category181708 Aloka Prosound SSD-3500 Plus Ul...,181708,US,"ebay template .header-wrapper,.subheader-...",Boards,category181708,category181708 Aloka Prosound SSD-3500 Plus Ul...,category181708 aloka prosound ssd3500 plus ult...,2


# Going in the fun

In [27]:
DF = df[~(df.group == "None")] 
X_train, X_test, y_train, y_test = train_test_split(df['Title_Clean'], 
                                                    df['Category_Code'], 
                                                    test_size=0.15, 
                                                    random_state=8)

In [28]:
# Parameter election
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

In [29]:
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

(19680, 300)
(3474, 300)


In [30]:
from sklearn.feature_selection import chi2
import numpy as np

for Product, category_id in sorted(category_codes.items()):
    features_chi2 = chi2(features_train, labels_train == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}' category:".format(Product))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-2:])))
    print("")

# 'Array' category:
  . Most correlated unigrams:
. array
. ultrasonic
. transducer
. ge
. category184515
  . Most correlated bigrams:
. ultrasound probe
. ultrasound transducer

# 'Bladder' category:
  . Most correlated unigrams:
. 3000
. bladderscan
. bvi
. verathon
. bladder
  . Most correlated bigrams:
. bvi 3000
. bladder scanner

# 'Boards' category:
  . Most correlated unigrams:
. 5000
. hdi
. assy
. pcb
. board
  . Most correlated bigrams:
. atl hdi
. hdi 5000

# 'Cables' category:
  . Most correlated unigrams:
. new
. use
. brand
. medical
. series
  . Most correlated bigrams:
. probe transducer
. brand new

# 'Chips' category:
  . Most correlated unigrams:
. ultrasound
. mhz
. hitachi
. module
. category181909
  . Most correlated bigrams:
. ultrasound probe
. ultrasound transducer

# 'Doppler' category:
  . Most correlated unigrams:
. hp
. ultraschallgerät
. mhz
. category117028
. doppler
  . Most correlated bigrams:
. category3179 atl
. transducer ultrasound

# 'Fetal' categ

In [31]:
if 1:
    pathPickle = "Pickles"
else:
    pathPickle = "Pickles_FullDesc"
    
with open(pathPickle+'/df.pickle', 'wb') as output:
    pickle.dump(df, output)
    
with open(pathPickle+'/X_train.pickle', 'wb') as output:
    pickle.dump(X_train, output)
    
# X_test    
with open(pathPickle+'/X_test.pickle', 'wb') as output:
    pickle.dump(X_test, output)
    
# y_train
with open(pathPickle+'/y_train.pickle', 'wb') as output:
    pickle.dump(y_train, output)
    
# y_test
with open(pathPickle+'/y_test.pickle', 'wb') as output:
    pickle.dump(y_test, output)
    
# df
with open(pathPickle+'/df.pickle', 'wb') as output:
    pickle.dump(df, output)
    
    
# features_train
with open(pathPickle+'/features_train.pickle', 'wb') as output:
    pickle.dump(features_train, output)

# labels_train
with open(pathPickle+'/labels_train.pickle', 'wb') as output:
    pickle.dump(labels_train, output)

# features_test
with open(pathPickle+'/features_test.pickle', 'wb') as output:
    pickle.dump(features_test, output)

# labels_test
with open(pathPickle+'/labels_test.pickle', 'wb') as output:
    pickle.dump(labels_test, output)
    
# TF-IDF object
with open(pathPickle+'/tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)
    
    
with open(pathPickle+'/category_codes.pickle', 'wb') as output:
    pickle.dump(category_codes, output)
